# A Keras Convolutional Neural Network on CIFAR-10



https://github.com/fchollet/keras/blob/master/examples/cifar10_cnn.py

In [2]:
import tensorflow as tf 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

import keras
from keras.datasets import cifar10
from keras.models import Sequential 
from keras.models import model_from_json
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

## Data acquisition & hyperparameter setting

In [3]:
batch_size = 32
num_classes = 10
epochs = 1

In [4]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print("y_train shape:", y_train.shape)

y_train shape: (50000, 10)


## Build model 

In [6]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
__________

## Compile Model 

model compilation 

In [9]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

optimizers 

In [ ]:
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

compile 

In [10]:
#
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
x_train.shape

(50000, 32, 32, 3)

model fitting 

In [11]:
model.fit(x_train[:20000], y_train[:20000],
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

Not using data augmentation.
Train on 20000 samples, validate on 10000 samples
Epoch 1/1
20000/20000 [==============================] - 289s - loss: 2.0332 - acc: 0.2501 - val_loss: 1.8250 - val_acc: 0.3569

## Retrieve weights from Layers 

common practice get weights 

In [12]:
print(len(model.layers[0].get_weights()))
print(model.layers[0].get_weights()[0].shape)
print(model.layers[0].get_weights()[1].shape)
conv1_weights = model.layers[0].get_weights()[0]
conv1_biases = model.layers[0].get_weights()[1]
model.layers[0].get_weights()[0]

2
(3, 3, 3, 32)
(32,)


array([[[[ 0.0556595 ,  0.09106687,  0.0952794 , -0.07818365,  0.05960592,
           0.13091703,  0.14947316,  0.13326134, -0.10681936, -0.10567041,
           0.12215956,  0.0802748 ,  0.04450392,  0.00048477,  0.11889506,
           0.03521046, -0.03748699, -0.08963996,  0.02083039,  0.12819295,
           0.04273948,  0.13970801, -0.01928986, -0.03323125,  0.05116474,
           0.00274411,  0.05294781, -0.06317707, -0.00190593,  0.08520786,
           0.03848021, -0.01069758],
         [ 0.02940613, -0.02856179, -0.03034916,  0.06506472,  0.06601148,
           0.0202391 , -0.11831344,  0.01285229, -0.07415377, -0.11608568,
           0.04489761,  0.12055461,  0.13345225, -0.10011332, -0.12304367,
          -0.10735974, -0.11884381,  0.08546261,  0.05049191,  0.0410815 ,
          -0.03770501, -0.04237143,  0.06747782,  0.03644151,  0.10892386,
           0.11004978,  0.01887542, -0.12749888, -0.01290551, -0.03063028,
          -0.05524983, -0.12101046],
         [-0.07929356, -0.

## Save the model & trained weights 

save model formats ... 

In [16]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


## Reload model & weights 

reload 

In [ ]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")